In [2]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import norm
from  scipy.stats import binom
from scipy.stats import beta
from  scipy.stats import uniform
from scipy.stats import gaussian_kde

import pystan
from pystan.constants import MAX_UINT

%run ../tools.py

# Exercise 1

The goal of this exercise is to implement the Metropolis-Hastings method as seen in Lesson 5. Consider a variable y_i representing the number of success of an experiment i repeated n times. Now consider a vector Y containing the number of success of N independent experiments y_i. We assume that each experiment_i share a common probability of success $\theta = 0.3$. Therefore we can generate the following data:

In [2]:
N = 100
n = 100
theta = 0.3

Y = binom.rvs(n, theta, size=N)
    
print(Y[:10])

[29 37 32 28 30 34 22 31 27 31]


**1) We will assume a prior Beta distribution with parameters a=0.5 and b=0.5 for $\theta$. Write the data likelihood p(Y|$\theta$) and the posterior distribution p($\theta$ | Y).**

**2) Estimate the posterior distirbution p($\theta$|Y) using the Metropolis-Hastings algorithm.**

In [4]:
def proposal(prop_mu, prop_sigma):
    return scipy.stats.norm.rvs(loc = prop_mu, scale = prop_sigma, size = 1)

def log_likelihood(param):
    return np.sum([binom.logpmf(y_i,n,param) for y_i in Y])

def log_prior(param):
    return beta.logpdf(param, 0.5, 0.5)

def log_posterior(param):
    return log_likelihood(param) + log_prior(param)

**3) Plot the posterior probability density functions obtained with the MCMC approximation and the analytical solution.**

# Exercise 2

A farmer owns a huge amount of cows. This year he ran an experiment  where he gave 10 cows medicine A and 10 medicine B and then measured whether they got sick (0) or not (1) during the summer season. Here is the resulting data:

In [97]:
cowA = np.array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0])
cowB = np.array([0, 0, 1, 1, 1, 0, 1, 1, 1, 0])

**1) The farmer wants to know: How effective are the drugs? What is the evidence that medicine A is better or worse than medicine B ?**

Hint: The outcome for each cow taking medecine A can be modeled as a random variable following a Bernouilli distribution with parameter $\theta_1$. Similarly, the outcome for each cow taking medecine B can be modeled as a random variable following a Bernouilli distribution with parameter $\theta_2$.

The farmer has a huge number of cows. Earlier this year he ran an experiment where he gave 10 cows a special diet that he had heard could make them produce more milk. He recorded the number of liters of milk from these “diet” cows and from 15 “normal” cows during one month. This is the data:

In [113]:
diet_milk = np.array([651., 679., 374., 601., 401., 609., 767., 709., 704., 679.])
normal_milk = np.array([798., 1139., 529., 609., 553., 743., 151., 544., 488., 555., 257., 692., 678., 675., 538.])

**2) The farmer now wants to know: Was the diet any good, does it results in better milk production? **

Hint: The most common approach here would be to model the milk production of each cow as a normal distribution. 

The farmer also has chickens. He tries different diets on them too with the hope that they will produce more eggs. Below is the number of eggs produced in one week by chickens on a diet and chickens eating normal chicken food:

In [2]:
diet_eggs = np.array([6, 4, 2, 3, 4, 3, 0, 4, 0, 6, 3])
normal_eggs =  np.array([4, 2, 1, 1, 2, 1, 2, 1, 3, 2, 1])

**3) The farmer now wants to know: Was the diet any good, does it result in the chickens producing more eggs ? ** 

The farmer is now wondering whether the amount of time a cow spends outside in the sunshine affects how much milk she produces. To test this he makes a controlled experiment where he picks out 20 cows and assigns each a number of hours she should spend outside each day. The experiment runs for a month and Jöns records the number of liters of milk each cow produces. The data is the following:

In [3]:
milk = np.array([685, 691, 476, 1151, 879, 725, 1190, 1107, 809, 539, 298, 805, 820, 498, 1026, 1217, 1177, 684, 1061, 834])
hours = np.array([3, 7, 6, 10, 6, 5, 10, 11, 9, 3, 6, 6, 3, 5, 8, 11, 12, 9, 5, 5])

**4)  Using this data on hours of sunshine and resulting liters of milk the farmer wants to know: Does sunshine affect milk production positively or negatively? **

# Exercise 3

Weakly informative priors are especially critical when inferences are hindered with only weakly identifiable likelihoods, such as those arising from models with sparse data.

To that end, let’s say that we are analyzing a small company and we want to model how much daily rainfall, x, affects daily income, y, using only a few measurements. For this study we will simulate data assuming that the company typically makes a few thousand dollars, or kilodollars (k$), per day without any rain and that a heavy rainfall of a few centimeters per day can severely curtail income.

In [87]:
alpha = 1    # k$
beta = -0.25 # k$ / cm
sigma = 1    # k$

N = 5
rain = 2.*np.random.rand(N) # cm
daily_income = np.random.normal(loc=beta * x + alpha, scale=sigma) # k$

print(rain)
print(daily_income)

[0.8689176  1.1808266  1.89158407 1.23102573 0.5816685 ]
[ 0.40355738 -0.90128158  1.52170117  1.66406968  1.87581803]


Assuming that the typical values of both rainfall and income are sufficiently large, we can ignore the fact that they are positive quantities and model their relationship with a linear regression. We can then fit this linear regression in Stan using a very long Markov chain to ensure precise quantification of our posterior distribution.

**1) Fit a linear regression model using Stan and study the posterior distribution of your parameters. You will use a flat prior (this is done automatically by Stan if you don't specify any prior on your parameters.)** 

## Weakly Informative Priors

Weakly informative priors introduce scale information to regularize inferences. Scales are straightforward to reason about in applied problems, especially when units are carefully laid out, and they provide just enough information to regularize non-identified or weakly-identified likelihoods without strongly biasing the posterior away from reasonable parameter values. In order to construct weakly informative priors we need to first decompose our model into components, define default values, identify scales, then choose an explicit shape for our prior.

We cannot define scales, let alone reason about them, until we first decompose our model into interpretable components. In other words, we need to find a parameterization of our model where the parameters are particularly meaningful. The parameterization we have used in our linear regression, for example, is ideal as the intercept, slope, and measurement variability have intuitive interpretations: the intercept, $\alpha$, determines the base income without any rainfall, the slope, $\beta$, controls how a change in rainfall affects income, and the measurement variation, $\sigma$, quantifies the natural variability of daily income.

## Weakly Informative Priors Under Well-Chosen Scales

### Light-Tailed Weakly Informative Priors

When the scales are well-chosen all weakly informative priors behave similarly, regularizing the posterior by penalizing extreme parameter values. The exact shape of a weakly informative prior, however, does introduce some important differences in how strong that regularization is. Let’s first consider a relatively light-tailed weakly informative prior that utilizes Gaussian distributions. Because we simulated the data already in natural units, the weakly informative priors are given simply by unit-scale Gaussians.

**2) Fit a linear regression model using Stan. You will use a standard Gaussian as prior for all the parameters.** 

**3) Plot the histogram for the posterior distribution of the parameters. Compare with the ground truth. What do you observe compared to question 1) ? **

### Heavy-Tailed Weakly Informative Priors

To contrast, let’s now consider the more heavily-tailed priors given by Cauchy distributions. Once again our prescient choice of units admits unit-scale distributions.

**4) Fit a linear regression model using Stan. You will use a standard Cauchy as prior for all the parameters.** 

**5) Plot the histogram for the posterior distribution of the parameters. Compare with the ground truth. What do you observe compared to question 3) ? **

## Weakly Informative Priors Under Poorly-Chosen Scales

The differing behavior of weakly informative priors with different shapes becomes particularly striking when the scales are poorly-chosen, and the likelihood strongly favors parameter values that conflict with the priors.

To demonstrate this difference, let’s simulate a larger data set where the true intercept is above the scale of our chosen units. How this tension between the weakly informative prior and the likelihood manifests in the posterior is extremely sensitive to the exact shape of the weakly informative prior.

In [71]:
alpha = 10   # k$
beta = -0.25 # k$ / cm
sigma = 1    # k$

N = 10
rain = 2.*np.random.rand(N) # cm
daily_income = np.random.normal(loc=beta * rain + alpha, scale=sigma) # k$

print(rain)
print(daily_income)

[1.93558116 0.24031392 1.73401905 0.47031533 0.09843725 1.24823894
 0.69594907 1.74843803 0.16011238 0.42924275]
[11.4851514   9.42848573  8.23247025  9.36580534 10.98259069 10.86842408
  8.8595824  10.08770919 10.50106175  9.81588199]


### The Failure Mode of Light Tails

Let’s first consider the relatively light tail of a Gaussian prior distribution.

**6) Fit a linear regression model using Stan. You will use a standard Gaussian as prior for all the parameters.** 

**7) Plot the histogram for the posterior distribution of the parameters. Compare with the ground truth. What do you observe compared to question 3) ? **

### Heavy-Tailed Weakly Informative Priors

The heavier tail of the Cauchy prior responds very differently to a poorly-chosen scale.

**8) Fit a linear regression model using Stan. You will use a standard Cauchy as prior for all the parameters.** 

**9) Plot the histogram for the posterior distribution of the parameters. Compare with the ground truth. What do you observe compared to question 7) ? **